# PROJECT ONE 

# MY TOP QUERIES

## *MEDIUM QUERY*
**PROPOSITION**: Create or update a view named Purchasing.SupplierPerformanceStats to store summarized performance metrics for suppliers based on their historical 

**DATABASE**: WideWorldimporters Database

**TABLES**: Purchasing.Suppliers, Purchasing.PurchaseOrders, Purchasing.PurchaseOrderLines

**COLUMNS**: SupplierID, SupplierName, OrderDate, ExpectedDeliveryDate, ExpectedUnitPricePerOuter, OrderedOuters

**VIEW**: Purchasing.SupplierPerformanceStats view

**ATTRIBUTES**: SupplierID, SupplierName, AverageLeadTimeDays, TotalPurchaseAmount

**PREDICATE**: This query selects supplier performance metrics by calculating the average lead time for order fulfillment and the total purchase amount spent on orders from the Purchasing.Suppliers, Purchasing.PurchaseOrders, and Purchasing.PurchaseOrderLines tables. It computes the average lead time in days and the total purchase amount for each supplier. The data is filtered to include only purchase orders placed between January 1, 2013, and December 31, 2013. These aggregated metrics are stored or updated in the Purchasing.SupplierPerformanceStats view. Finally, it selects SupplierName, AverageLeadTimeDays, and TotalPurchaseAmount columns from the view, ordered by TotalPurchaseAmount in descending order.

**TOP QUERY REASON**:
1. *Efficient Aggregation and Calculations*- It aggregates key performance indicators for suppliers, such as average lead time and total purchase amount, efficiently. These metrics are vital for assessing supplier reliability and their financial contribution over a specified period.
2. *Relevance to Business Decisions*- The metrics calculated are directly relevant to supply chain management, helping businesses identify reliable suppliers and optimize supplier relationships based on performance data.
3. *Clarity and Readability* -With a structured and neat format, including the use of a Common Table Expression (CTE), the query is easily understandable, enhancing its maintainability and modularity for future adjustments.





In [6]:
USE WideWorldimporters; -- MEDIUM QUERY
GO

WITH SupplierPerformance AS (
    SELECT
        s.SupplierID,
        s.SupplierName,
        AVG(DATEDIFF(day, po.OrderDate, po.ExpectedDeliveryDate)) AS AverageLeadTimeDays,
        SUM(pol.ExpectedUnitPricePerOuter * pol.OrderedOuters) AS TotalPurchaseAmount
    FROM
        Purchasing.Suppliers s
    INNER JOIN Purchasing.PurchaseOrders po ON s.SupplierID = po.SupplierID
    INNER JOIN Purchasing.PurchaseOrderLines pol ON po.PurchaseOrderID = pol.PurchaseOrderID
    WHERE
        po.OrderDate BETWEEN '2013-01-01' AND '2013-12-31'
    GROUP BY
        s.SupplierID, s.SupplierName
)
SELECT
    SupplierName,
    AverageLeadTimeDays,
    TotalPurchaseAmount
FROM
    SupplierPerformance
ORDER BY
    TotalPurchaseAmount DESC
FOR JSON PATH, ROOT('SupplierPerformance');


Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.374

JSON_F52E2B61-18A1-11d1-B105-00805F49916B
"{""SupplierPerformance"":[{""SupplierName"":""Fabrikam, Inc."",""AverageLeadTimeDays"":18,""TotalPurchaseAmount"":53015178.00},{""SupplierName"":""Litware, Inc."",""AverageLeadTimeDays"":19,""TotalPurchaseAmount"":8617182.30},{""SupplierName"":""Northwind Electric Cars"",""AverageLeadTimeDays"":16,""TotalPurchaseAmount"":78816.50},{""SupplierName"":""The Phone Company"",""AverageLeadTimeDays"":18,""TotalPurchaseAmount"":50820.00},{""SupplierName"":""Graphic Design Institute"",""AverageLeadTimeDays"":16,""TotalPurchaseAmount"":6489.00},{""SupplierName"":""Contoso, Ltd."",""AverageLeadTimeDays"":14,""TotalPurchaseAmount"":313.50}]}"


## *COMPLEX QUERY*
**PROPOSITION**: Develop or update a function named dbo.CalculateTotalCost within the PrestigeCars database. This function computes the comprehensive cost for a vehicle, integrating the initial cost with the expenses incurred from repairs, parts, and transport.

**DATABASE**: PrestigeCars

**TABLE**: Data.Stock, Data.Model, Data.Make, Data.SalesDetails

**COLUMN**: Data.Stock.Cost, Data.Stock.StockCode, Data.Stock.ModelID, Data.Model.ModelID, Data.Model.ModelName, Data.Model. MakeID, Data.Make.MakeID, Data.Make.MakeName, Data.SalesDetails.SalePrice, Data.SalesDetails.StockID, Data.SalesDetails.SalesID 

**FUNCTION**: dbo.CalculateTotalCost

**PARAMETERS**:
- @Cost (MONEY) - Specifies the initial cost of the vehicle.
- @RepairsCost (MONEY) - Indicates the expenses associated with vehicle repairs.
- @PartsCost (MONEY) - Denotes the cost of any parts required for the vehicle.
- @TransportInCost (MONEY) - Represents the cost involved in transporting the vehicle.

**RETURN TYPE**: MONEY - The aggregate cost of the vehicle, including its initial cost and subsequent expenses, returned as a monetary value.

**PREDICATE**: Utilizing a Common Table Expression (CTE) named CarSalesAnalysis, the query aggregates comprehensive cost data for vehicles, applying dbo.CalculateTotalCost to combine the initial cost with repair, parts, and transport expenses. Following this, another CTE, SalesSummary, is employed to collate this cost data alongside sales information, calculating total costs, total sales, average sale price, and the number of sales per make and model. The output is methodically grouped by make and model, with the final selection presenting these summarized attributes, prioritizing the records with the highest total costs in descending order.

**TOP QUERY REASON**:
1. *Custom Function Integration* -
Utilizes a custom function to dynamically calculate total costs, showcasing sophisticated SQL capabilities for real-world applications.
2. *Holistic Data Analysis* -
Combines data from multiple sources to provide comprehensive insights into car sales, demonstrating efficient handling of complex relational data.
3. *Strategic Insights through Advanced Aggregation* -
Employs advanced SQL features for detailed data aggregation and analysis, offering key metrics critical for informed decision-making.



In [ ]:
Use PrestigeCars -- COMPLEX QUERY
GO
CREATE OR ALTER FUNCTION dbo.CalculateTotalCost(
    @Cost money,
    @RepairsCost money,
    @PartsCost money,
    @TransportInCost money
)
RETURNS money
AS
BEGIN
    RETURN @Cost + @RepairsCost + @PartsCost + @TransportInCost;
END;
GO

WITH CarSalesAnalysis AS (
    SELECT
        MK.MakeName,
        MD.ModelName,
        dbo.CalculateTotalCost(ST.Cost, ST.RepairsCost, ST.PartsCost, ST.TransportInCost) AS TotalCost,
        SD.SalePrice
    FROM Data.Stock ST
    INNER JOIN Data.Model MD ON ST.ModelID = MD.ModelID
    INNER JOIN Data.Make MK ON MD.MakeID = MK.MakeID
    INNER JOIN Data.SalesDetails SD ON ST.StockCode = SD.StockID
),
SalesSummary AS (
    SELECT
        MakeName,
        ModelName,
        SUM(TotalCost) AS TotalCosts,
        SUM(SalePrice) AS TotalSales,
        AVG(SalePrice) AS AverageSalePrice,
        COUNT(*) AS NumberOfSales
    FROM CarSalesAnalysis
    GROUP BY MakeName, ModelName
)
SELECT MakeName, ModelName, TotalCosts, TotalSales, AverageSalePrice, NumberOfSales
FROM SalesSummary
FOR JSON PATH, ROOT('SalesSummary');



## *MEDIUM QUERY*
**PROPOSITION**: Create or revise a function named dbo.CalculateTotalCost in the PrestigeCars database. This function determines the total cost for a vehicle by incorporating the base sale price and any additional charges.

**DATABASE**: PrestigeCars

**FUNCTION**: dbo.CalculateTotalCost

**PARAMETERS**:
- @SalePrice (MONEY) - Indicates the base sale price of the vehicle.
- @AdditionalCharges (FLOAT) - Represents any extra charges applied to the sale (e.g., delivery, service plans).

**RETURN TYPE**: MONEY - The total cost of the vehicle, combining both the sale price and additional charges, returned as a monetary value.

**PREDICATE**: The query uses a Common Table Expression (CTE) named CarCostSummary to aggregate cost data from SalesDetails for vehicles sold in a specific year. By leveraging dbo.CalculateTotalCost, it calculates the total cost and average additional charges per make and model. The results are organized by make, model, and year. The concluding query selects summarized attributes, arranged by the highest total cost in descending order.


**TOP QUERY REASON:** 

1. *Custom Calculation*- Uses a custom function to precisely calculate total vehicle costs.
2. *Data Integration*-  Merges data from several tables to offer a complete view of car sales.
3. *Insightful Analysis*- Employs aggregation to provide essential sales metrics, aiding strategic decisions.

In [ ]:
USE PrestigeCars;
GO

-- CTE to aggregate sales by Make and Model
WITH CarSalesSummary AS (
    SELECT
        MK.MakeName,
        MD.ModelName,
        SUM(SD.SalePrice) AS TotalSales,
        AVG(SD.SalePrice) AS AverageSalePrice,
        COUNT(SD.SalesID) AS NumberOfSales
    FROM Data.Sales SA
    INNER JOIN Data.SalesDetails SD ON SA.SalesID = SD.SalesID
    INNER JOIN Data.Stock ST ON SD.StockID = ST.StockCode
    INNER JOIN Data.Model MD ON ST.ModelID = MD.ModelID
    INNER JOIN Data.Make MK ON MD.MakeID = MK.MakeID
    GROUP BY MK.MakeName, MD.ModelName
)
-- Select the data in relational format
SELECT MakeName, ModelName, TotalSales, AverageSalePrice, NumberOfSales
FROM CarSalesSummary

FOR JSON PATH, ROOT('CarSalesSummary');

# MY WORST QUERIES

## *MEDIUM QUERY*
**PROPOSITION**: Construct or refine a query within the PrestigeCars database utilizing the [Data].[SalesByCountry] view. This query is aimed at summarizing sales figures by country, focusing on total sales value, average sale price, and the number of transactions.

**DATABASE**: PrestigeCars

**VIEW**: [Data].[SalesByCountry]

**PREDICATE**: Employing a Common Table Expression (CTE) named SalesSummary, the query aggregates sales information from the [Data].[SalesByCountry] view. It calculates the total sales value (considering sale price and line item discounts), the average sale price, and counts the distinct number of transactions (invoices) for each country. The grouped results are then presented by country name, with summaries of the total sales value, average sale price, and the number of transactions. This analysis provides a concise overview of sales performance segmented by country, showcasing the critical sales metrics in a structured format.

**WORST QUERY REASON:**
1. *Performance Efficiency:*- It may process large datasets inefficiently without date range filters.
2. *Analysis Depth:* - Lacks granularity for detailed insights by summarizing only at the country level.
3. *Insight Breadth:* - Focuses on basic metrics, missing the opportunity to explore more comprehensive sales dynamics.

**HOW TO MAKE IT BETTER:**
To refine the query, incorporating date range filters could significantly improve performance by narrowing down the data to a more relevant subset. Expanding the analysis to include more granular details, such as regional breakdowns within countries or temporal trends, would offer deeper insights. Additionally, broadening the range of metrics to analyze, like sales variability, customer demographics, or product performance, could provide a more comprehensive understanding of sales dynamics, thereby making the query not only more efficient but also more insightful and actionable for strategic decision-making.



In [ ]:
USE PrestigeCars;
GO

-- Query using the view [Data].[SalesByCountry] to summarize sales by country
WITH SalesSummary AS (
    SELECT
        CountryName,
        SUM(SalePrice - LineItemDiscount) AS TotalSalesValue,
        AVG(SalePrice - LineItemDiscount) AS AverageSalePrice,
        COUNT(DISTINCT InvoiceNumber) AS NumberOfTransactions
    FROM [Data].[SalesByCountry]
    GROUP BY CountryName
)
SELECT
    CountryName,
    TotalSalesValue,
    AverageSalePrice,
    NumberOfTransactions
FROM SalesSummary

FOR JSON PATH, ROOT('SalesSummary');


## *MEDIUM QUERY*
**PROPOSITION**: Formulate or update a query in the WideWorldImportersDW database to compute and summarize monthly purchases from suppliers. This involves calculating both the total and average quantity of items ordered from each supplier, segmented by month and year.

**DATABASE**: WideWorldImportersDW

**PREDICATE**: The query leverages a Common Table Expression (CTE) named MonthlySupplierPurchases to consolidate purchase data from the Fact.Purchase table, in conjunction with the Dimension.Date and Dimension.Supplier tables. It aggregates the data to calculate the total and average quantity of items ordered from suppliers, grouping the results by calendar month, year, and supplier. The output is then organized to display these calculations along with the corresponding month, year, and supplier name. This approach provides a detailed overview of purchasing activity with suppliers over time, facilitating analysis of purchasing trends and supplier performance.

**WORST QUERY REASON:**
1. *Lack of Date Filtering:* Without recent data filtering, it might analyze outdated information, affecting relevance and performance.
2. *Oversimplified Grouping:* The broad monthly grouping by supplier without product detail can miss nuanced insights.
3. *Variable Averages:* Averaging order quantities without segmentation can lead to misleading conclusions due to high variability.

**HOW TO MAKE IT BETTER:**
To enhance the query, I would consider incorporating date range filters to focus on recent and relevant data, thereby improving performance and insight applicability. Adding seperation by product type or category within the supplier grouping can offer more detailed and actionable insights. Introducing more granular metrics, such as median quantities or categorizing suppliers by volume, could also reduce the variability in averages, providing a clearer understanding of supplier performance and improving decision-making based on the query's outcomes.

In [ ]:
Use WideWorldImportersDW
;WITH MonthlySupplierPurchases AS (
    SELECT
        d.[Calendar Month Label] AS Month,
        d.[Calendar Year] AS Year,
        s.Supplier,
        SUM(p.[Ordered Quantity]) AS TotalQuantity,
        AVG(p.[Ordered Quantity]) AS AverageQuantity
    FROM
        Fact.Purchase AS p
        JOIN Dimension.Date AS d ON p.[Date Key] = d.Date
        JOIN Dimension.Supplier AS s ON p.[Supplier Key] = s.[Supplier Key]
    GROUP BY
        d.[Calendar Month Label],
        d.[Calendar Year],
        s.Supplier
)
-- Relational output
SELECT Month, Year, Supplier, TotalQuantity, AverageQuantity
FROM MonthlySupplierPurchases

FOR JSON PATH, ROOT('MonthlySupplierPurchases');


## _MEDIUM QUERY_

**PROPOSITION**: Construct or enhance a query within the AdventureWorksDW2017 database to analyze the average list price of products, segmented by product category and subcategory. This query focuses on providing insights into pricing strategies across different segments of the product hierarchy.

**DATABASE**: AdventureWorksDW2017

**PREDICATE**: Utilizing a Common Table Expression (CTE) named ProductPriceSummary, the query systematically aggregates product pricing data from the dbo.DimProduct table, alongside hierarchical categorization from dbo.DimProductSubcategory and dbo.DimProductCategory tables. By filtering out products without a list price, it calculates the average list price within each product category and subcategory. The results are grouped accordingly, offering a refined view that facilitates understanding of average pricing across various product segments. This strategic grouping provides valuable insights for pricing analysis, marketing strategies, and product placement decisions.

**OUTPUT FORMATS**:

- Standard SQL output: Delivers a relational representation that enumerates the product categories, subcategories, and their corresponding average list prices, aiding in straightforward analysis and comparison across different product segments.
- JSON output (commented): Offers an alternate output format by structuring the data into a JSON document labeled 'ProductPriceSummary'. This format is conducive for scenarios requiring structured data exchange, integration with web applications, or further processing in data analytics platforms that support JSON.

**WORST QUERY REASON:**
1. *Exclusion of Free Products* - Ignoring items with a list price of 0 might overlook important product strategy insights.
2. *Narrow Pricing Analysis:* - Focusing only on average list prices limits the understanding of broader pricing dynamics within categories.
3. *Output Format Constraints:* - The exclusive use of JSON for output may not align with the preferences or needs of all users, hindering data accessibility and integration with other analysis tools.

**HOW TO MAKE IT BETTER:**
Enhancing the query involves including products with a list price of 0 to ensure a holistic analysis of pricing strategies and product breadth. Expanding the analysis to encompass a wider range of pricing metrics—such as variability, median prices, and their sales correlations—will offer deeper insights into market dynamics and pricing effectiveness. Additionally, providing output in both JSON and standard relational formats will accommodate diverse user needs, making the query's insights more accessible for various analytical processes and applications, thus maximizing its utility and applicability across different analysis scenarios.

In [ ]:
Use AdventureWorksDW2017

-- Calculating average list price by product category and subcategory
;WITH ProductPriceSummary AS (
    SELECT
        pc.EnglishProductCategoryName AS CategoryName,
        psc.EnglishProductSubcategoryName AS SubcategoryName,
        AVG(p.ListPrice) AS AverageListPrice
    FROM dbo.DimProduct AS p
    INNER JOIN dbo.DimProductSubcategory AS psc ON p.ProductSubcategoryKey = psc.ProductSubcategoryKey
    INNER JOIN dbo.DimProductCategory AS pc ON psc.ProductCategoryKey = pc.ProductCategoryKey
    WHERE p.ListPrice > 0 -- Excluding products with no list price
    GROUP BY pc.EnglishProductCategoryName, psc.EnglishProductSubcategoryName
)
-- Relational output
SELECT CategoryName, SubcategoryName, AverageListPrice
FROM ProductPriceSummary

-- JSON output
FOR JSON PATH, ROOT('ProductPriceSummary');


# _MEDIUM QUERIES_

**PROPOSITION**: Design or refine a query in the AdventureWorks2017 database for the purpose of summarizing sales data by customer. This summary includes the total number of orders, the total sales value, and the average value of orders placed by each customer.

**DATABASE**: AdventureWorks2017

**PREDICATE**: The query utilizes a Common Table Expression (CTE) named CustomerSalesSummary to amalgamate sales data across customers, leveraging joins between the Sales.Customer, Sales.SalesOrderHeader, and Sales.SalesOrderDetail tables. It computes the total number of orders per customer, the cumulative sales value, and the average order value, with a grouping based on the CustomerID. This aggregation facilitates a comprehensive view of customer engagement and spending behavior, highlighting customer contribution to sales revenue.

**OUTPUT FORMATS**:

- Standard SQL output: Provides a relational display of each customer's ID along with their respective total number of orders, total sales, and average order value. This format aids in understanding individual customer contributions and identifying high-value customers.
- JSON output (commented): Offers an alternate data representation, encapsulating the sales summary for customers in a JSON structure tagged as 'CustomerSalesSummary'. This format is ideal for data interchange or processing in web applications and services, ensuring a structured and easily accessible format for further analysis or reporting.

In [ ]:
Use AdventureWorks2017
;WITH CustomerSalesSummary AS (
    SELECT
        c.CustomerID,
        COUNT(soh.SalesOrderID) AS TotalOrders,
        SUM(sod.LineTotal) AS TotalSales,
        AVG(sod.LineTotal) AS AverageOrderValue
    FROM Sales.Customer c
    INNER JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    INNER JOIN Sales.SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID
    GROUP BY c.CustomerID
)
-- Relational output
SELECT CustomerID, TotalOrders, TotalSales, AverageOrderValue
FROM CustomerSalesSummary

-- JSON output
FOR JSON PATH, ROOT('CustomerSalesSummary');


## _MEDIUM QUERY_

**PROPOSITION**: Develop or enhance a query within the AdventureWorks2017 database to aggregate sales data by customer. This query focuses on computing the total number of orders, the total sales value, and the average order value for each customer.

**DATABASE**: AdventureWorks2017

**PREDICATE**: Utilizing a Common Table Expression (CTE) named CustomerSalesSummary, the query consolidates sales data by linking the Sales.Customer, Sales.SalesOrderHeader, and Sales.SalesOrderDetail tables. It computes the total number of orders, total sales value, and average order value for each customer, filtering the data based on customer IDs. he final output is structured to display the customer ID along with their total orders, total sales, and average order value, thereby providing a detailed account of individual customer contributions to overall sales.

**OUTPUT FORMATS**:

- Standard SQL output: Offers a straightforward, relational view that lists each customer's ID, total number of orders, total sales, and average order value, facilitating an easy assessment of customer value and behavior.
- JSON output (commented): Suggests an alternate representation of the summarized sales data in a JSON format, encapsulated under the label 'CustomerSalesSummary'. This structured format is ideal for scenarios requiring data interchange or further processing in JSON-compatible systems, ensuring versatility in data application and analysis.

In [ ]:
Use Northwinds2022TSQLV7
;WITH CustomerOrderSummary AS (
    SELECT
        c.CustomerCompanyName,
        COUNT(DISTINCT o.OrderId) AS TotalOrders,
        SUM(od.UnitPrice * od.Quantity) AS TotalSalesValue,
        SUM(od.Quantity) AS TotalProductsOrdered
    FROM Sales.[Order] o
    INNER JOIN Sales.Customer c ON o.CustomerId = c.CustomerId
    INNER JOIN Sales.OrderDetail od ON o.OrderId = od.OrderId
    GROUP BY c.CustomerCompanyName
)
-- Relational output
SELECT CustomerCompanyName, TotalOrders, TotalSalesValue, TotalProductsOrdered
FROM CustomerOrderSummary

-- JSON output
 FOR JSON PATH, ROOT('CustomerOrderSummary');


## *MEDIUM QUERY*
**PROPOSITION**: Design or modify a query in the Northwinds2022TSQLV7 database to compile a summary of orders by customer company. This query aims to detail the total number of distinct orders, the aggregate sales value, and the total quantity of products ordered, all categorized by customer company name.

**DATABASE**: Northwinds2022TSQLV7

**PREDICATE**: The query employs a Common Table Expression (CTE) named CustomerOrderSummary to aggregate and analyze order data across the Sales.Order, Sales.Customer, and Sales.OrderDetail tables. It meticulously calculates the total number of distinct orders, the cumulative sales value, and the total quantity of products ordered for each customer company. This aggregation is segmented by customer company name, ensuring a clear and organized presentation of sales data that highlights the purchasing behavior and financial contribution of each customer company to the business. The summarized data is then showcased, providing vital insights into customer company performance through metrics such as total orders, total sales value, and total products ordered.

**OUTPUT FORMATS**:
- Standard SQL output: Presents a relational summary displaying the customer company name alongside their total number of orders, total sales value, and total products ordered. This format facilitates an immediate understanding of each company's business activities and their significance to overall sales.
- JSON output (commented): Offers an alternative data representation, structuring the summarized order information in a JSON format, labeled 'CustomerOrderSummary'. This approach is particularly suited for digital environments requiring data serialization, enabling easy data transfer and integration with systems capable of processing JSON.

In [ ]:
Use Northwinds2022TSQLV7
;WITH SupplierProductSummary AS (
    SELECT
        s.SupplierCompanyName,
        COUNT(p.ProductId) AS TotalProducts,
        AVG(p.UnitPrice) AS AverageUnitPrice
    FROM Production.Product p
    INNER JOIN Production.Supplier s ON p.SupplierId = s.SupplierId
    GROUP BY s.SupplierCompanyName
)
-- Relational output
SELECT SupplierCompanyName, TotalProducts, AverageUnitPrice
FROM SupplierProductSummary

-- -- JSON output

FOR JSON PATH, ROOT('SupplierProductSummary');


## *MEDIUM QUERY*
**PROPOSITION**: Formulate or adjust a query in the WideWorldImportersDW database to synthesize sales data per customer, incorporating the city context. This endeavor aims to present both the total and average sales amounts, uniquely identifying each customer by their key and including the city for further geographical insights.

**DATABASE**: WideWorldImportersDW

**PREDICATE**: Through a Common Table Expression (CTE) named SalesByCustomer, the query amalgamates sales information by linking the Dimension.Customer, Fact.Sale, and Dimension.City tables. It aims to calculate the total and average sales amounts for each customer, ensuring each record is distinctly grouped by the customer key, customer name, and their associated city. This detailed grouping fosters a comprehensive understanding of customer sales patterns, integrating geographical data for a richer analysis. By presenting the customer key, customer name, city, total sales, and average sale amount, the query offers a nuanced view of customer sales activity, enhancing strategic business decisions based on geographical sales performance.

**OUTPUT FORMATS**:
- Standard SQL output: Provides a relational table showcasing the customer key, customer name, city, total sales, and average sale amount. This format is instrumental for direct, actionable insights into customer behavior and sales trends across different geographical locations.
- JSON output (commented): Proposes an alternative format, encoding the sales summary data in JSON, tagged 'SalesByCustomer'. This structured format caters to scenarios requiring digital data exchange or integration with systems designed to process JSON, facilitating flexible data manipulation and visualization.

In [ ]:
Use WideWorldImportersDW
;WITH SalesByCustomer AS (
    SELECT
        c.[Customer Key],
        c.Customer,
        ci.City,
        SUM(s.[Total Including Tax]) AS TotalSales,
        AVG(s.[Total Including Tax]) AS AverageSaleAmount
    FROM
        Dimension.Customer AS c
        JOIN Fact.Sale AS s ON c.[Customer Key] = s.[Customer Key]
        JOIN Dimension.City AS ci ON s.[City Key] = ci.[City Key]
    GROUP BY
        c.[Customer Key],
        c.Customer,
        ci.City
)
-- Relational output
SELECT [Customer Key], Customer, City, TotalSales, AverageSaleAmount
FROM SalesByCustomer

-- JSON output
FOR JSON PATH, ROOT('SalesByCustomer');


## *MEDIUM QUERY*
**PROPOSITION**: Create or update a view named Sales.CustomerSalesSummary to store aggregated sales data for customer categories based on their purchases. Then, select and display CustomerCategoryName, TotalSales, and AverageSalePerInvoice for each customer category from the Sales.CustomerSalesSummary view.

**DATABASE**: WideWorldimporters Database

**TABLES**: Sales.Customers, Sales.Invoices, Sales.InvoiceLines, Sales.CustomerCategories

**COLUMNS**: CustomerID, CustomerCategoryID, InvoiceDate, InvoiceLineID, ExtendedPrice, CustomerCategoryName

**VIEW**: Sales.CustomerSalesSummary view

**ATTRIBUTES**: CustomerCategoryName, TotalSales, AverageSalePerInvoice

**PREDICATE**: This query computes aggregated sales data for different customer categories by joining the Sales.Customers, Sales.Invoices, Sales.InvoiceLines, and Sales.CustomerCategories tables. It extracts customer ID, customer category ID, invoice date, invoice line ID, and extended price from the Sales.Customers, Sales.Invoices, and Sales.InvoiceLines tables, focusing on invoices issued between January 1, 2014, and December 31, 2014. The data is aggregated to calculate the total sales and average sale per invoice for each customer category. These aggregated metrics are stored or updated in the Sales.CustomerSalesSummary view. Finally, it selects CustomerCategoryName, TotalSales, and AverageSalePerInvoice columns from the view, ordered by TotalSales in descending order.
<u></u>






In [ ]:
USE WideWorldimporters; -- Medium
GO

WITH CustomerSales AS (
    SELECT
        c.CustomerID,
        c.CustomerCategoryID,
        i.InvoiceDate,
        il.InvoiceLineID,
        il.ExtendedPrice
    FROM
        Sales.Customers c
    JOIN Sales.Invoices i ON c.CustomerID = i.CustomerID
    JOIN Sales.InvoiceLines il ON i.InvoiceID = il.InvoiceID
    WHERE
        i.InvoiceDate BETWEEN '2014-01-01' AND '2014-12-31'
),
AggregatedSales AS (
    SELECT
        CustomerCategoryID,
        SUM(ExtendedPrice) AS TotalSales,
        AVG(ExtendedPrice) AS AverageSalePerInvoice
    FROM
        CustomerSales
    GROUP BY
        CustomerCategoryID
)
SELECT
    cc.CustomerCategoryName,
    asales.TotalSales,
    asales.AverageSalePerInvoice
FROM
    AggregatedSales asales
JOIN Sales.CustomerCategories cc ON asales.CustomerCategoryID = cc.CustomerCategoryID
ORDER BY
    TotalSales DESC

-- JSON output
FOR JSON PATH, ROOT('CustomerSales');


## *MEDIUM QUERY*

**PROPOSITION**: Create or update a view named Warehouse.StockSummary to store summarized information about stock items including their names, quantities on hand, and associated stock group names. Then, select and display StockItemName, QuantityOnHand, and StockGroupName for each stock item from the Warehouse.StockSummary view.

**DATABASE**: WideWorldimporters Database

**TABLES**: Warehouse.StockItemHoldings, Warehouse.StockItems, Warehouse.StockItemStockGroups, Warehouse.StockGroups

**COLUMNS**: StockItemID, StockItemName, QuantityOnHand, StockGroupName

**VIEW**: Warehouse.StockSummary view

**ATTRIBUTES**: StockItemName, QuantityOnHand, StockGroupName

**PREDICATE**: This query retrieves information about stock items from the Warehouse.StockItemHoldings, Warehouse.StockItems, Warehouse.StockItemStockGroups, and Warehouse.StockGroups tables. It selects the stock item ID, name, quantity on hand, and the associated stock group name. This data is then used to create or update the Warehouse.StockSummary view. Finally, it selects StockItemName, QuantityOnHand, and StockGroupName columns from the view, ordered by StockGroupName and then by StockItemName.

In [ ]:
USE WideWorldimporters;
GO

WITH StockSummary AS (
    SELECT
        si.StockItemID,
        si.StockItemName,
        sish.QuantityOnHand,
        sig.StockGroupName
    FROM
        Warehouse.StockItemHoldings sish
    JOIN Warehouse.StockItems si ON sish.StockItemID = si.StockItemID
    JOIN Warehouse.StockItemStockGroups sisg ON si.StockItemID = sisg.StockItemID
    JOIN Warehouse.StockGroups sig ON sisg.StockGroupID = sig.StockGroupID
)
SELECT
    StockItemName,
    QuantityOnHand,
    StockGroupName
FROM
    StockSummary
ORDER BY
    StockGroupName, StockItemName

-- FOR JSON PATH, ROOT('StockSummary');


# *COMPLEX QUERIES*
**PROPOSITION**: Create or update a view named Sales.RegionalTopSellingProducts to store summarized sales data by region and top-selling products for the year 2013. Then, select and display Region, StockItem, and TotalSales for the top 5 selling products in each region from the Sales.RegionalTopSellingProducts view.

**DATABASE**: WideWorldimportersDW Database

**TABLES**: Fact.Sale, Dimension.StockItem, Dimension.City, Dimension.Date

**COLUMNS**: Region, Stock Item, Total Including Tax, Stock Item Key, City Key, Invoice Date Key, Calendar Year

**VIEW**: Sales.RegionalTopSellingProducts view

**ATTRIBUTES**: Region, StockItem, TotalSales

**PREDICATE**: This query summarizes sales data by region and product category for the year 2013 using a Common Table Expression (CTE) named RegionalSalesData. It then ranks the sales data within each region to identify the top-selling products using another CTE named RankedRegionalSales. The top 5 selling products in each region are then selected and ordered by region and sales rank. This query creates or updates the Sales.RegionalTopSellingProducts view with this aggregated data. Finally, it selects Region, Stock Item, and TotalSales columns from the view to display the top-selling products in each region.

In [ ]:
USE WideWorldimportersDW; -- COMPLEX 
GO

-- Define a CTE to summarize sales data by region and product category in 2020
WITH RegionalSalesData AS (
    SELECT
        ci.Region,
        si.[Stock Item],
        SUM(fs.[Total Including Tax]) AS TotalSales
    FROM
        Fact.Sale fs
    INNER JOIN Dimension.[Stock Item] si ON fs.[Stock Item Key] = si.[Stock Item Key]
    INNER JOIN Dimension.City ci ON fs.[City Key] = ci.[City Key]
    INNER JOIN Dimension.Date d ON fs.[Invoice Date Key] = d.Date
    WHERE
        d.[Calendar Year] = 2013
    GROUP BY
        ci.Region,
        si.[Stock Item]
),
-- Rank sales data within each region to identify top-selling products
RankedRegionalSales AS (
    SELECT
        Region,
        [Stock Item],
        TotalSales,
        RANK() OVER (PARTITION BY Region ORDER BY TotalSales DESC) AS SalesRank
    FROM
        RegionalSalesData
)
-- Select the top 5 selling products in each region
SELECT
    Region,
    [Stock Item],
    TotalSales
FROM
    RankedRegionalSales
WHERE
    SalesRank <= 5
ORDER BY
    Region, 
    SalesRank

-- FOR JSON PATH, ROOT('RegionalSalesData');

## *COMPLEX QUERY*
**PROPOSITION**<span style="color: var(--vscode-foreground);">: Establish or revise a function called dbo.GetTotalInvoiceTax within the WideWorldImporters database. The function is designed to compute the total tax amount for a given invoice and return it as a decimal value with precision up to 18 digits and 2 decimal places. The function queries the Sales.InvoiceLines table, sums up the TaxAmount column values where the InvoiceID matches the provided @InvoiceID parameter, and stores the result in a variable named @TotalTax. This computed total tax amount is then returned by the function.</span>  

**DATABASE**: WideWorldImporters

**FUNCTION**: dbo.GetTotalInvoiceTax

**RETURN TYPE**: DECIMAL(18,2)

**PARAMETER**: @InvoiceID is an integer parameter utilized by dbo.GetTotalInvoiceTax function to identify the InvoiceID for which the total tax amount is to be calculated.

**PREDICATE**: The function dbo.GetTotalInvoiceTax calculates the total tax amount for a given invoice by summing up the TaxAmount column from the Sales.InvoiceLines table where the InvoiceID matches the input parameter @InvoiceID. The result is returned as a decimal value with precision up to 18 digits and 2 decimal places.

In [ ]:
USE WideWorldImporters --HARD
GO
CREATE OR ALTER FUNCTION  dbo.GetTotalInvoiceTax(@InvoiceID INT)
RETURNS DECIMAL(18,2)
AS
BEGIN
    DECLARE @TotalTax DECIMAL(18,2);
    SELECT @TotalTax = SUM(TaxAmount) FROM Sales.InvoiceLines WHERE InvoiceID = @InvoiceID;
    RETURN @TotalTax;
END;
GO

;WITH InvoiceSummary AS (
    SELECT
        i.InvoiceID,
        i.CustomerID,
        i.InvoiceDate,
        dbo.GetTotalInvoiceTax(i.InvoiceID) AS TotalTax,
        SUM(il.ExtendedPrice) AS TotalExtendedPrice,
        COUNT(il.InvoiceLineID) AS NumberOfItems
    FROM Sales.Invoices i
    INNER JOIN Sales.InvoiceLines il ON i.InvoiceID = il.InvoiceID
    GROUP BY i.InvoiceID, i.CustomerID, i.InvoiceDate
),
StockTransactions AS (
    SELECT
        sit.StockItemID,
        sit.Quantity,
        sit.TransactionOccurredWhen,
        sit.InvoiceID
    FROM Warehouse.StockItemTransactions sit
    WHERE sit.InvoiceID IS NOT NULL
)
SELECT top (5)
    ISum.InvoiceID,
    ISum.CustomerID,
    ISum.InvoiceDate,
    ISum.TotalTax,
    ISum.TotalExtendedPrice,
    ISum.NumberOfItems,
    ST.Quantity AS TransactionQuantity,
    ST.TransactionOccurredWhen
FROM InvoiceSummary ISum
INNER JOIN StockTransactions ST ON ISum.InvoiceID = ST.InvoiceID
ORDER BY ISum.InvoiceDate DESC, ISum.InvoiceID

FOR JSON PATH, ROOT('dbo.GetTotalInvoiceTax');


## *COMPLEX QUERY*
**PROPOSITION**: Develop or update a function named dbo.AvgCostPerItem in the WideWorldImporters database. This function is designed to compute the average cost per item for a specified purchase order.

**DATABASE**: WideWorldImporters

**FUNCTION**: dbo.AvgCostPerItem

**PARAMETER**: @PurchaseOrderID (INT) - Used to specify the PurchaseOrderID for which the average cost per item is calculated.

**RETURN TYPE**: DECIMAL(18,2) - The average cost per item is returned as a decimal value with precision up to 18 digits and 2 decimal places.

**PREDICATE**: The function dbo.AvgCostPerItem is created or updated within the WideWorldImporters database to facilitate the computation of the average cost per item for a specified purchase order. It achieves this by internally calculating the mean value of the ExpectedUnitPricePerOuter column from the Purchasing.PurchaseOrderLines table, filtered based on the provided PurchaseOrderID parameter. This function encapsulates the logic required to perform the calculation, ensuring consistency and ease of use. Additionally, it promotes modularization and abstraction, allowing for better code organization and maintenance.

In [ ]:
USE WideWorldImporters; --HARD
GO

CREATE OR ALTER FUNCTION dbo.AvgCostPerItem(@PurchaseOrderID INT)
RETURNS DECIMAL(18,2)
AS
BEGIN
    RETURN (
        SELECT AVG(ExpectedUnitPricePerOuter)
        FROM Purchasing.PurchaseOrderLines
        WHERE PurchaseOrderID = @PurchaseOrderID
    );
END;
GO

;WITH PurchaseOrderDetails AS (
    SELECT
        pol.PurchaseOrderID,
        pol.StockItemID,
        SUM(pol.OrderedOuters) AS TotalOrderedOuters,
        SUM(pol.ReceivedOuters) AS TotalReceivedOuters
    FROM Purchasing.PurchaseOrderLines pol
    GROUP BY pol.PurchaseOrderID, pol.StockItemID
), StockSummary AS (
    SELECT
        sih.StockItemID,
        sih.QuantityOnHand,
        AVG(sih.LastCostPrice) AS AvgLastCostPrice -- Assuming AVG is meaningful here
    FROM Warehouse.StockItemHoldings sih
    GROUP BY sih.StockItemID, sih.QuantityOnHand
)
SELECT top (5)
    po.PurchaseOrderID,
    po.SupplierID,
    po.OrderDate,
    POD.TotalOrderedOuters,
    POD.TotalReceivedOuters,
    dbo.AvgCostPerItem(po.PurchaseOrderID) AS AvgCostPerItem,
    SS.QuantityOnHand,
    SS.AvgLastCostPrice
FROM Purchasing.PurchaseOrders po
INNER JOIN PurchaseOrderDetails POD ON po.PurchaseOrderID = POD.PurchaseOrderID
LEFT JOIN StockSummary SS ON POD.StockItemID = SS.StockItemID
ORDER BY po.OrderDate DESC

FOR JSON PATH, ROOT('dbo.AvgCostPerItem');

## *COMPLEX QUERY*
**PROPOSITION**: Create or update a function named dbo.CalculateAdjustedSalesAmount in the WideWorldImportersDW database. This function increases the total sales, including tax, by 10%.

**DATABASE**: WideWorldImportersDW

**FUNCTION**: dbo.CalculateAdjustedSalesAmount

**PREDICATE**: The function dbo.CalculateAdjustedSalesAmount adjusts the total sales amount, including tax, by a fixed percentage increase of 10%.

The query utilizes a Common Table Expression (CTE) named EmployeeSalesPerformance to aggregate sales data for employees in 2013. It computes the adjusted sales amount for each employee using the dbo.CalculateAdjustedSalesAmount function. The CTE selects relevant attributes such as Employee Key, Employee name, Calendar Year, and AdjustedSalesAmount.

Finally, the main query selects Employee, CalendarYear, and AdjustedSalesAmount from the CTE, ordering the results by AdjustedSalesAmount in descending order.

In [ ]:
USE WideWorldImportersDW; --Hard
GO

-- Create or alter a custom function to calculate adjusted sales amount
CREATE OR ALTER FUNCTION dbo.CalculateAdjustedSalesAmount(@totalSalesIncludingTax DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    -- Example adjustment: a simple 10% increase to account for performance metrics
    RETURN @totalSalesIncludingTax * 1.1;
END;
GO

WITH EmployeeSalesPerformance AS (
    SELECT
        e.[Employee Key],
        e.Employee,
        d.[Calendar Year],
        SUM(dbo.CalculateAdjustedSalesAmount(s.[Total Including Tax])) AS AdjustedSalesAmount
    FROM
        Fact.Sale s
    INNER JOIN Dimension.Employee e ON s.[Salesperson Key] = e.[Employee Key]
    INNER JOIN Dimension.Date d ON s.[Invoice Date Key] = d.[Date]
    WHERE
        d.[Calendar Year] = 2013
    GROUP BY
        e.[Employee Key],
        e.[Employee],
        d.[Calendar Year]
)
SELECT
    Employee
    CalendarYear,
    AdjustedSalesAmount
FROM
    EmployeeSalesPerformance
ORDER BY
    AdjustedSalesAmount DESC


## *COMPLEX QUERY*
**PROPOSITION**: Introduce or update a function named dbo.CalculateNetSales in the AdventureWorksDW2017 database. This function calculates the net sales amount based on the provided sales amount and discount amount.

**DATABASE**: AdventureWorksDW2017

**FUNCTION**: dbo.CalculateNetSales

**PARAMETERS**: 
- @SalesAmount (MONEY) - Represents the total sales amount.
- @DiscountAmount (FLOAT) - Represents the discount amount applied to the sales.

**RETURN TYPE**: MONEY - The net sales amount returned as a monetary value.

**PREDICATE**: The query employs a Common Table Expression (CTE) named SalesSummary to aggregate sales data from FactInternetSales for 2013. Utilizing dbo.CalculateNetSales, it computes net sales and average discounts per product category. The results are grouped by category name and year. The final query selects summarized attributes, ordered by total net sales in descending order.

In [ ]:
USE AdventureWorksDW2017;-- Hard 
GO

CREATE OR ALTER FUNCTION dbo.CalculateNetSales(
    @SalesAmount MONEY,
    @DiscountAmount FLOAT
)
RETURNS MONEY
AS
BEGIN
    RETURN @SalesAmount - (@SalesAmount * @DiscountAmount);
END;
GO

WITH SalesSummary AS (
    SELECT
        pc.EnglishProductCategoryName AS CategoryName,
        d.CalendarYear,
        SUM(dbo.CalculateNetSales(fs.SalesAmount, fs.DiscountAmount)) AS TotalNetSales,
        AVG(fs.DiscountAmount) AS AverageDiscount
    FROM
        dbo.FactInternetSales fs
    INNER JOIN dbo.DimProduct p ON fs.ProductKey = p.ProductKey
    INNER JOIN dbo.DimProductCategory pc ON p.ProductSubcategoryKey = pc.ProductCategoryKey
    INNER JOIN dbo.DimDate d ON fs.OrderDateKey = d.DateKey
    WHERE
        d.CalendarYear = 2013
    GROUP BY
        pc.EnglishProductCategoryName,
        d.CalendarYear
)
SELECT
    CategoryName,
    CalendarYear,
    TotalNetSales,
    AverageDiscount
FROM
    SalesSummary
ORDER BY
    TotalNetSales DESC

--FOR JSON PATH;
FOR JSON PATH, ROOT('dbo.CalculateNetSales');


## *COMPLEX QUERY*
**PROPOSITION**: Establish or revise a function named dbo.AdjustSalesAmount within the AdventureWorksDW2017 database. This function adjusts sales amounts based on the month of the year.

**DATABASE**: AdventureWorksDW2017

**FUNCTION**: dbo.AdjustSalesAmount

**PARAMETERS**: 
- @SalesAmount (MONEY) - Represents the original sales amount.
- @MonthNumber (TINYINT) - Indicates the month number for which the adjustment is applied.

**RETURN TYPE**: MONEY - Returns the adjusted sales amount.

**PREDICATE**: The dbo.AdjustSalesAmount function facilitates the adjustment of sales amounts depending on the month of the year. It applies a specific adjustment factor to sales amounts based on the provided month number. Subsequently, the query utilizes two Common Table Expressions (CTEs) to analyze sales data by category and month for the year 2013. The first CTE, MonthlySalesByCategory, aggregates sales data, adjusting sales amounts using the dbo.AdjustSalesAmount function and grouping by product category, month, and year. The second CTE, AverageMonthlySalesByCategory, computes the average monthly sales and total products sold for each product category. Finally, the main query selects CategoryName, CalendarYear, AverageMonthlySales, and TotalProductsSold from the AverageMonthlySalesByCategory CTE, ordered by AverageMonthlySales in descending order.

In [ ]:
USE AdventureWorksDW2017; --Hard
GO

CREATE OR ALTER FUNCTION dbo.AdjustSalesAmount(
    @SalesAmount MONEY,
    @MonthNumber TINYINT
)
RETURNS MONEY
AS
BEGIN
    DECLARE @AdjustmentFactor DECIMAL(5, 2) = CASE 
        WHEN @MonthNumber BETWEEN 6 AND 8 THEN 1.10 -- Example: Increase summer sales by 10%
        ELSE 1.00 -- No adjustment outside summer
    END;
    RETURN @SalesAmount * @AdjustmentFactor;
END;
GO
WITH MonthlySalesByCategory AS (
    SELECT
        pc.EnglishProductCategoryName AS CategoryName,
        MONTH(d.FullDateAlternateKey) AS MonthNumber,
        d.CalendarYear,
        SUM(dbo.AdjustSalesAmount(fs.SalesAmount, MONTH(d.FullDateAlternateKey))) AS AdjustedSalesAmount,
        COUNT(DISTINCT fs.ProductKey) AS ProductsSold
    FROM
        dbo.FactInternetSales fs
    INNER JOIN dbo.DimDate d ON fs.OrderDateKey = d.DateKey
    INNER JOIN dbo.DimProduct p ON fs.ProductKey = p.ProductKey
    INNER JOIN dbo.DimProductCategory pc ON p.ProductSubcategoryKey = pc.ProductCategoryKey
    WHERE
        d.CalendarYear = 2013
    GROUP BY
        pc.EnglishProductCategoryName,
        MONTH(d.FullDateAlternateKey),
        d.CalendarYear
),
AverageMonthlySalesByCategory AS (
    SELECT
        CategoryName,
        CalendarYear,
        AVG(AdjustedSalesAmount) AS AverageMonthlySales,
        SUM(ProductsSold) AS TotalProductsSold
    FROM
        MonthlySalesByCategory
    GROUP BY
        CategoryName,
        CalendarYear
)
SELECT
    CategoryName,
    CalendarYear,
    AverageMonthlySales,
    TotalProductsSold
FROM
    AverageMonthlySalesByCategory
ORDER BY
    AverageMonthlySales DESC

--FOR JSON PATH;
FOR JSON PATH, ROOT('dbo.AdjustSalesAmount');

## COMPLEX QUERY
**PROPOSITION**: Create or update a function named dbo.CategorizeSalesPerformance within the Northwinds2022TSQLV7 database. This function categorizes sales performance based on total sales amount.

**DATABASE**: Northwinds2022TSQLV7

**FUNCTION**: dbo.CategorizeSalesPerformance

**PARAMETERS**: 
- @TotalSales (MONEY) - Represents the total sales amount for a product.

**RETURN TYPE**: NVARCHAR(50) - Returns a string indicating the sales performance category ('High', 'Medium', or 'Low').

**PREDICATE**: The dbo.CategorizeSalesPerformance function simplifies the assessment of sales performance by categorizing total sales amounts into predefined levels. The subsequent query utilizes a Common Table Expression (CTE) named ProductSales to aggregate sales data for products. It calculates the total sales amount for each product, grouped by category name and product name. Another CTE named SalesPerformance applies the dbo.CategorizeSalesPerformance function to categorize sales performance based on total sales. Finally, the main query selects CategoryName, ProductName, TotalSales, and Performance for analysis, ordered by CategoryName and TotalSales for clarity.

In [ ]:
USE Northwinds2022TSQLV7; --HARD
GO

CREATE OR ALTER FUNCTION dbo.CategorizeSalesPerformance(@TotalSales MONEY)
RETURNS NVARCHAR(50)
AS
BEGIN
    RETURN CASE 
        WHEN @TotalSales > 10000 THEN 'High'
        WHEN @TotalSales BETWEEN 5000 AND 10000 THEN 'Medium'
        ELSE 'Low'
    END;
END;
GO

WITH ProductSales AS (
    SELECT
        pc.CategoryName,
        p.ProductName,
        SUM(od.Quantity * od.UnitPrice) AS TotalSales
    FROM
        Production.Product p
    JOIN Production.Category pc ON p.CategoryID = pc.CategoryID
    JOIN Sales.OrderDetail od ON p.ProductID = od.ProductID
    GROUP BY
        pc.CategoryName,
        p.ProductName
),
SalesPerformance AS (
    SELECT
        CategoryName,
        ProductName,
        TotalSales,
        dbo.CategorizeSalesPerformance(TotalSales) AS Performance
    FROM
        ProductSales
)
SELECT
    CategoryName,
    ProductName,
    TotalSales,
    Performance
FROM
    SalesPerformance
ORDER BY
    CategoryName, TotalSales DESC

--FOR JSON PATH;
FOR JSON PATH, ROOT('dbo.CategorizeSalesPerformance');


## COMPLEX QUERY
**PROPOSITION**: Introduce or modify a function named dbo.EvaluateRestockingPriority within the AdventureWorks2017 database. This function evaluates the restocking priority of inventory items based on their current stock levels and a specified restock threshold.

**DATABASE**: AdventureWorks2017

**FUNCTION**: dbo.EvaluateRestockingPriority

**PARAMETERS**: 
- @CurrentStockLevel (INT) - Represents the current stock level of the inventory item.
- @RestockThreshold (INT) - Indicates the threshold value at which restocking is deemed necessary.

**RETURN TYPE**: NVARCHAR(10) - Returns a string indicating the restocking priority ('High', 'Medium', or 'Low').

**PREDICATE**: The dbo.EvaluateRestockingPriority function simplifies inventory management by determining the restocking priority for items based on their current stock levels compared to a specified threshold. The subsequent query utilizes a Common Table Expression (CTE) named StockAnalysis to analyze stock levels of products categorized by subcategory and location. It applies dbo.EvaluateRestockingPriority to evaluate the restocking priority for each item. The results include attributes such as SubcategoryName, ProductName, LocationName, StockLevel, and RestockingPriority, providing valuable insights into inventory management. The query orders the results by SubcategoryName, RestockingPriority, and StockLevel for improved clarity.

In [ ]:
USE AdventureWorks2017;-- HARD
GO

CREATE OR ALTER FUNCTION dbo.EvaluateRestockingPriority(
    @CurrentStockLevel INT,
    @RestockThreshold INT
)
RETURNS NVARCHAR(10)
AS
BEGIN
    RETURN CASE 
        WHEN @CurrentStockLevel < @RestockThreshold THEN 'High'
        WHEN @CurrentStockLevel BETWEEN @RestockThreshold AND (@RestockThreshold * 2) THEN 'Medium'
        ELSE 'Low'
    END;
END;
GO
WITH StockAnalysis AS (
    SELECT
        ps.Name AS SubcategoryName,
        p.Name AS ProductName,
        l.Name AS LocationName,
        pi.Quantity AS StockLevel,
        dbo.EvaluateRestockingPriority(pi.Quantity, 100) AS RestockingPriority -- Assuming 100 as a generic restock threshold
    FROM
        Production.ProductInventory pi
    INNER JOIN Production.Product p ON pi.ProductID = p.ProductID
    INNER JOIN Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
    INNER JOIN Production.Location l ON pi.LocationID = l.LocationID
)
SELECT
    SubcategoryName,
    ProductName,
    LocationName,
    StockLevel,
    RestockingPriority
FROM
    StockAnalysis
ORDER BY
    SubcategoryName, RestockingPriority, StockLevel

--FOR JSON PATH;
FOR JSON PATH, ROOT('dbo.EvaluateRestockingPriority');

